In [ ]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! python -m spacy download ru_core_news_lg

2022-09-02 18:03:26.659676: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 513.4 MB 12 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


In [ ]:
import re

In [ ]:
import math
import random
import string

import numpy as np
import seaborn as sns
import pandas as pd

import torch
import nltk
import gensim
import gensim.downloader as api

In [ ]:
!gdown https://drive.google.com/uc?id=1mmEKwO7vtaPoEVfmmwDC9X7vMY3KYS1a&export=download
data = pd.read_csv("test_data.csv")

Downloading...
From: https://drive.google.com/uc?id=1mmEKwO7vtaPoEVfmmwDC9X7vMY3KYS1a
To: /content/test_data.csv
100% 53.4k/53.4k [00:00<00:00, 54.3MB/s]


In [ ]:
greetings = '''Hello!	Здравствуйте!
Hi!	Привет!
Good morning!	Доброе утро!
Good afternoon!	Добрый день!
Good evening!	Добрый вечер!
Welcome!	Добро пожаловать!
Goodbye!	До свидания!
See you later!	До встречи!
See you soon!	До скорого!
Good luck!	Счастливо!
Have a good trip!	Счастливого пути!
Good night!	Спокойной ночи!'''

In [ ]:
greetings = greetings.lower().replace('\t', '').replace('\n', '').split('!')[1::2]

In [ ]:
greetings.append('всего доброго')

In [ ]:
hello, bye = greetings[:6], greetings[6:]

In [ ]:
data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [ ]:
data_lower = data.copy()
data_lower['text'] = data_lower['text'].apply(lambda x: x.lower())

In [ ]:
greetings

['здравствуйте',
 'привет',
 'доброе утро',
 'добрый день',
 'добрый вечер',
 'добро пожаловать',
 'до свидания',
 'до встречи',
 'до скорого',
 'счастливо',
 'счастливого пути',
 'спокойной ночи',
 'всего доброго']

In [ ]:
def find_greets(text, greetings, hello, bye):
  for word in greetings:
    if word in text and word in hello:
      return 1
    if word in text and word in bye:
      return 2
  return 0

In [ ]:
dict_for_greets = {1: 'Приветствие', 2: 'Прощание'}
data_lower['greets'] = data_lower[data_lower['role']=='manager']['text'].apply(lambda x: find_greets(x, greetings, hello, bye))

In [ ]:
dlg_id = data_lower.dlg_id.unique()
data_manag = data_lower[data_lower['role']=='manager']

In [ ]:
managers = pd.DataFrame({
    'dlg_id': dlg_id,
    'Hello':0,
    'Hello_text':'',
    'Bye':0,
    'Bye_text':'',
    'PER':'',
    'ORG':''
})
for dlg in dlg_id:
  if 1 in data_manag[data_manag['dlg_id']==dlg]['greets'].to_numpy():
    ind_hello = data_manag[(data_manag['dlg_id']==dlg)&(data_manag['greets']==1)]['text'].to_string()
    managers.loc[dlg,'Hello'] = 'Да'
    managers.loc[dlg,'Hello_text'] = ind_hello
  if 2 in data_manag[data_manag['dlg_id']==dlg]['greets'].to_numpy():
    ind_bye = data_manag[(data_manag['dlg_id']==dlg)&(data_manag['greets']==2)]['text'].to_string()
    managers.loc[dlg, 'Bye'] = 'Да'
    managers.loc[dlg,'Bye_text'] = ind_bye
managers

,dlg_id,Hello,Hello_text,Bye,Bye_text,PER,ORG
0,0,Да,1 алло здравствуйте,Да,108 всего хорошего до свидания,,
1,1,Да,110 алло здравствуйте,Да,163 до свидания,,
2,2,Да,166 алло здравствуйте,0,,,
3,3,Да,250 алло дмитрий добрый день,Да,300 угу все хорошо да понедельника тогда вс...,,
4,4,0,,Да,335 во вторник все ну с вами да тогда до вт...,,
5,5,0,,Да,479 ну до свидания хорошего вечера,,


In [ ]:
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words("russian")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words.append('алло')

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
data_lower.head()

,dlg_id,line_n,role,text,greets
0,0,0,client,алло,NaN
1,0,1,manager,алло здравствуйте,1.0
2,0,2,client,добрый день,NaN
3,0,3,manager,меня зовут ангелина компания диджитал бизнес з...,0.0
4,0,4,client,ага,NaN


In [ ]:
for ix, line in data_lower.iterrows():
  new_text = []
  text = line['text']
  tokens = word_tokenize(text, language="russian")
  for token in tokens:
    if token not in stop_words:
      new_text.append(token)
  data_lower.loc[ix, 'text'] = ' '.join(new_text)

In [ ]:
data_lower

,dlg_id,line_n,role,text,greets
0,0,0,client,,NaN
1,0,1,manager,здравствуйте,1.0
2,0,2,client,добрый день,NaN
3,0,3,manager,зовут ангелина компания диджитал бизнес звоним...,0.0
4,0,4,client,ага,NaN
...,...,...,...,...,...
475,5,138,manager,поводу виджетов ними обсудите конкретно продам,0.0
476,5,139,manager,высылаю счет вами связи будут вопросы можете п...,0.0
477,5,140,client,спасибо спасибо,NaN
478,5,141,client,созвонимся ага спасибо давайте,NaN


In [ ]:
import spacy
from spacy import displacy
from collections import Counter

In [ ]:
import ru_core_news_lg
nlp = ru_core_news_lg.load()

In [ ]:
data_start = data_lower[data_lower['line_n']<10]

In [ ]:
for ix, line in data_lower.iterrows():
  doc = nlp(line['text'])
  for named_entity in doc.ents:
    if named_entity.label_ == 'PER':
        print(line['dlg_id'],named_entity ) 
      

0 ангелина
0 хирам
0 э
1 ангелина
1 ватсап
1 денис дэ е эн
2 ангелина
2 кэшбэка кэшбэк
2 кэшбэка
2 ватсап
2 жесупов
3 дмитрий добрый
3 максим
3 китобизнес
3 дмитрий коллега
3 анастасия
4 айдар
5 дима вопросы
5 ла ла ла
5 анастасия слышно
5 хэлп
5 вячеслав
5 максим


In [ ]:
idx = []
present_id = -1
for ix, line in data.iterrows():
  id = line['dlg_id']
  doc = nlp(line['text'])
  pattern = '\w{2,}\sзовут\s\w{2,}\s'
  match = re.search(pattern, line['text'])
  for named_entity in doc.ents:
    if named_entity.label_ == 'PER':
      if match != None and id!= present_id:
        present_id = line['dlg_id']
        managers.loc[line['dlg_id'], 'PER'] = named_entity
        managers.loc[line['dlg_id'], 'Introduction'] = line['text']
        #print(line['text'],line['dlg_id'],named_entity ) 
      idx.append(ix)
      

In [ ]:
company = []
for i,line in data_lower.iterrows():
  pattern = '\w{2,}\sкомпан\w\w\s\w{2,}'
  match = re.search(pattern, line['text'])
  pattern2 = 'организац\w\w\s\w{2,}'
  match2 = re.search(pattern2, line['text'])
  if match != None:
    print(i,line['dlg_id'],match[0], '\t', line['text'])
  if match2 != None:
    print(i,line['dlg_id'],match2[0], '\t', line['text'])
    

3 0 ангелина компания диджитал 	 зовут ангелина компания диджитал бизнес звоним поводу продления лицензии серым скоро срок заканчивается
16 0 50 компаний которые 	 совершенно верно делает 50 компаний которые звонят
37 0 разных компаний поэтому 	 настроены подключены разных компаний поэтому
42 0 целом компании даю 	 просто извините свой подход немножко целом компании даю просто хорошую качественную обратную связь которую возможно
50 0 хорошая компания 110 	 это хорошая компания 110
74 0 та компания которая 	 та компания которая работает видит
82 0 это компания которая 	 счету это компания которая звонит
111 1 ангелина компания диджитал 	 зовут ангелина компания диджитал бизнес звоню поводу продления сели обратила внимание срок заканчивается
167 2 ангелина компания диджитал 	 зовут ангелина компания диджитал бизнес звоню поводу продления лицензии мастера вами сотрудничали видео
176 2 ваша компания называется 	 ваша компания называется
251 3 зовут компания китобизнес 	 добрый максим зовут

In [ ]:
data_lower.loc[176:177]

,dlg_id,line_n,role,text,greets
176,2,12,client,ваша компания называется,NaN
177,2,13,manager,диджитал бизнес,0.0


In [ ]:
data_lower.loc[273-5:273+5]

,dlg_id,line_n,role,text,greets
268,3,19,manager,подскажите пожалуйста просто подскажем какие п...,0.0
269,3,20,client,понял,NaN
270,3,21,client,,NaN
271,3,22,client,,NaN
272,3,23,client,транспортная компания,NaN
273,3,24,manager,транспортная компания интеграции сделаны канал...,0.0
274,3,25,client,смотрите планируем это настраивать понимаю это...,NaN
275,3,26,manager,будем прям настраивать серым систему внедрять ...,0.0
276,3,27,client,давайте знаете сделаем любом случае,NaN
277,3,28,manager,давайте знаете,0.0


In [ ]:
managers.loc[0,'ORG'] = 'диджитал'
managers.loc[1,'ORG'] = 'диджитал'
managers.loc[2,'ORG'] = 'диджитал'
managers.loc[3,'ORG'] = 'китобизнес'

In [ ]:
managers['Требовани_выполнено'] = 'Нет'
managers.loc[(managers['Hello']=='Да') & (managers['Bye']=='Да'),'Требовани_выполнено'] = 'Да'
managers.rename(columns = {
                           'PER' : 'Имя_менеджера', 
                           'ORG' : 'Компания',
                           'Hello': 'Было_приветствие',
                           'Hello_text':'Приветствие',
                           'Bye_text':'Прощание',
                           'Bye':'Было_прощание'
                           }, inplace = True)

In [ ]:
managers

,dlg_id,Было_приветствие,Приветствие,Было_прощание,Прощание,Имя_менеджера,Компания,Introduction,Требовани_выполнено
0,0,Да,1 алло здравствуйте,Да,108 всего хорошего до свидания,ангелина,диджитал,Меня зовут ангелина компания диджитал бизнес з...,Да
1,1,Да,110 алло здравствуйте,Да,163 до свидания,ангелина,диджитал,Меня зовут ангелина компания диджитал бизнес з...,Да
2,2,Да,166 алло здравствуйте,0,,ангелина,диджитал,Меня зовут ангелина компания диджитал бизнес з...,Нет
3,3,Да,250 алло дмитрий добрый день,Да,300 угу все хорошо да понедельника тогда вс...,максим,китобизнес,Добрый меня максим зовут компания китобизнес у...,Да
4,4,0,,Да,335 во вторник все ну с вами да тогда до вт...,,,NaN,Нет
5,5,0,,Да,479 ну до свидания хорошего вечера,,,NaN,Нет
